In [36]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import seaborn as sns
import math
from matplotlib.patches import Polygon as MplPolygon
import os

In [37]:

def swap_xy(t):
    return (t[1], t[0])
def retrive_df_from_json(data_to_retrive):
    # Initialize a list to hold flattened data
    flattened_data = []
    
    # Flatten the facade_loc data
    for cell_key, buildings in constant_values.items():
        for building_key, values in buildings.items():
            facade_loc_list = values[data_to_retrive]
            for idx, loc in enumerate(facade_loc_list):
                flattened_data.append({
                    'building_key': building_key,
                    'facade_key': idx,
                    data_to_retrive: loc
                })
    
    # Convert flattened data to DataFrame
    flattened_df = pd.DataFrame(flattened_data)
    
    # Pivot the DataFrame
    pivot_df = flattened_df.pivot_table(index='facade_key', columns='building_key', values=data_to_retrive, aggfunc='first')
    # Reset index to remove all indices
    pivot_df.reset_index(drop=True, inplace=True)
    
    # Rename columns to strip 'building_' prefix
    pivot_df.columns = [col.replace('building_', '') for col in pivot_df.columns]
    return pivot_df

In [38]:
def extract_result_entries(data):
    building_roof_data = {}
    
    building_facade_data = {}
    building_facade_original = {}
    
    building_facade_loc = {}
    building_facade_azimuth = {}
    
    for cell_key, cell_value in data.items():
        for building_key, building_value in cell_value.items():
            roof_dict = building_roof_data.setdefault(building_key, {})
            
            facade_dict = building_facade_data.setdefault(building_key, {})
            facade_dict_original = building_facade_original.setdefault(building_key, {})
            
            facade_loc_dict = building_facade_loc.setdefault(building_key, {})
            facade_azimuth_dict = building_facade_azimuth.setdefault(building_key, {})
            
            building_id = int(building_key.split('_')[1])
            
            for day_key, day_value in building_value.items():
                for hour_key, hour_value in day_value.items():
                    hour = int(hour_key.split('_')[1])  # Extract hour number
                    roof_entry = hour_value.get('roof', 0.0)
                    roof_original = hour_value.get('roof_original', 0.0)
                    roof_loc = hour_value.get('roof_loc', (0.0, 0.0))
                    
                    facade_entries = hour_value.get('facade', {})
                    facade_original = hour_value.get('facade_original', {})
                    facade_loc_entries = hour_value.get('facade_loc', {})
                    facade_azimuth_entries = hour_value.get('facade_azimuth', {})
                    
                    roof_dict[hour] = {'area': roof_entry, 'loc': roof_loc, 'original_roof' : roof_original}
                    
                    facade_dict[hour] = facade_entries # Store facade entries directly
                    # original facade
                    facade_dict_original[hour] = facade_original  # Store facade entries directly
    
    return building_roof_data, building_facade_data, building_facade_original

def transform_facade_areas(facade_entries):
    buildings = facade_entries.keys()
    # buildings_original = facade_original.keys()
    data = []
    # data_original = []
    for hour in range(24):
        row = {'hour': hour}
        # row2 = {'hour': hour}
        for building in buildings:
            facades = facade_entries[building].get(hour, {})
            # facades_original = facade_original[building].get(hour, {})
            for facade_num, facade_value in facades.items():
                column_name = f'{building}_facade_{facade_num}_area'
                row[column_name] = facade_value
            # for facade_num, facade_area in facades_original.items():
            #     column_name2 = f'{building}_facade_{facade_num}_area'
            #     row2[column_name2] = facade_area                
        data.append(row)
        # data_original.append(row2)
    df = pd.DataFrame(data)
    # df_original=pd.DataFrame(data_original)
    # df.fillna(df_original,inplace=True)
    # df.fillna('shaded', inplace=True)
    df.set_index('hour', inplace=True)
    df.columns = df.columns.str.replace(r'building_', '', regex=True)
    df.columns = df.columns.str.replace(r'facade_', '', regex=True)
    df.columns = df.columns.str.replace(r'_area', '', regex=True)
    return df


def transform_facade_locations(facade_loc_entries):
    buildings = facade_loc_entries.keys()
    data = []
    for hour in range(24):
        row = {'hour': hour}
        for building in buildings:
            facades_loc = facade_loc_entries[building].get(hour, {})
            for facade_num, facade_loc in facades_loc.items():
                column_name = f'{building}_{facade_num}_loc'
                row[column_name] = (facade_loc[0], facade_loc[1])
        data.append(row)
    df = pd.DataFrame(data)
    df.fillna(0, inplace=True)
    df.set_index('hour', inplace=True)
    df.columns = df.columns.str.replace(r'building_', '', regex=True)
    df.columns = df.columns.str.replace(r'_loc', '', regex=True)
    first_non_zero_values = df.apply(first_non_zero)#.iloc[0,:]
    # expanded_df = pd.DataFrame(first_non_zero_values.tolist()).T
    return first_non_zero_values
def first_non_zero(series):
    return series.loc[series != 0].iloc[0] if (series != 0).any() else None
def transform_facade_azimuths(facade_azimuth_entries):
    buildings = facade_azimuth_entries.keys()
    data = []
    for hour in range(24):
        row = {'hour': hour}
        for building in buildings:
            facades_azimuth = facade_azimuth_entries[building].get(hour, {})
            for facade_num, facade_azimuth in facades_azimuth.items():
                column_name = f'{building}_facade_{facade_num}_azimuth'
                row[column_name] = facade_azimuth
        data.append(row)
    df = pd.DataFrame(data)
    # df.fillna('shaded', inplace=True)
    df.set_index('hour', inplace=True)
    df.columns = df.columns.str.replace(r'building_', '', regex=True)
    df.columns = df.columns.str.replace(r'facade_', '', regex=True)
    df.columns = df.columns.str.replace(r'_azimuth', '', regex=True)
    selected_columns = {}
    for col in df.columns:
        start_value = col.split('_')[0]
        if start_value not in selected_columns:
            selected_columns[start_value] = col
    filtered_df = df[list(selected_columns.values())]
    filtered_df.columns = filtered_df.columns.str.split('_').str[0]
    first_non_zero_values = filtered_df.apply(first_non_zero)
    expanded_df = pd.DataFrame(first_non_zero_values.tolist()).T
    return expanded_df


In [39]:
def transform_roof_areas(roof_entries):
    buildings = roof_entries.keys()
    # buildings_original = facade_original.keys()
    data = []
    # data_original = []
    for hour in range(24):
        row = {'hour': hour}
        for building in buildings:
            roof = roof_entries[building].get(hour, {})
            for roof_key, roof_value in roof.items():
                column_name = f'{building}_roof'
                row[column_name] = roof['area']         
        data.append(row)
    df = pd.DataFrame(data)
    df.set_index('hour', inplace=True)
    df.columns = df.columns.str.replace(r'building_', '', regex=True)
    # df.columns = df.columns.str.replace(r'facade_', '', regex=True)
    # df.columns = df.columns.str.replace(r'_area', '', regex=True)
    return df

In [40]:
def convert_to_tuple(s):
    if pd.isna(s):
        return np.nan
    return tuple(map(float, s.strip('()').split(',')))

def plot_polygons(df_building, df_facade):
    fig, ax = plt.subplots()
    for idx, col in enumerate(df_building.columns):
        # Remove NaN values
        polygon_points = df_building[col].dropna().tolist()
        
        if len(polygon_points) >= 3:
            # Swap x and y here
            polygon_points = [(y, x) for x, y in polygon_points]
            
            polygon = MplPolygon(polygon_points, closed=True, fill=None, edgecolor='r')
            ax.add_patch(polygon)
            x, y = zip(*polygon_points)
            ax.plot(x, y)  # Plot points
            
            # Calculate centroid
            centroid_x = np.mean(x)
            centroid_y = np.mean(y)
            centroid = (centroid_x, centroid_y)
            
            # Annotate centroid with column index
            ax.annotate(str(col), centroid, color='red', weight='bold', ha='center', va='center')

    # plot building
    for row_num, row in df_facade.iterrows():
        for col in df_facade.columns:
            if isinstance(row[col], list) and len(row[col]) == 2:
                y, x = row[col]  # Swap x and y here
                plt.scatter(x, y, label=f'Facade {col}')
                plt.text(x, y, str(f'F {row_num}'), fontsize=12)
    
    ax.set_aspect('equal')
    plt.xlabel('lat')  # Swap labels as well
    plt.ylabel('long')  # Swap labels as well
    plt.title('Polygons for Each Column')
    plt.show()

In [41]:
files = [file for file in os.listdir('results') if '.json' in file and 'loc' not in file]
files

['1-1_results_original_code_time_0.033.json',
 '1-2_results_original_code_time_0.080.json']

In [42]:
# for file in files: 
#     with open(f'results/{file}', 'r') as json_file:
#     constant_values = json.load(json_file)
#     pivot_df_building = retrive_df_from_json('roof_loc')
#     pivot_df_facade = retrive_df_from_json('facade_loc')
#     plot_polygons(pivot_df_building, pivot_df_facade)

In [43]:
for file in files: 
    test_case = file.split('_')[0]
    with open(f'results/{file}', 'r') as file:
        data = json.load(file)
    building_roof_data, building_facade_data, building_facade_original = extract_result_entries(data)
    facade_areas_df = transform_facade_areas(building_facade_data)
    facade_areas_df_original = transform_facade_areas(building_facade_original)
    roof_areas_df = transform_roof_areas(building_roof_data)
    combined_df = pd.concat([roof_areas_df, facade_areas_df], axis=1)
    sorted_columns = sorted(combined_df.columns, key=lambda x: int(x.split('_')[0]))
    combined_df = combined_df[sorted_columns]
    combined_df.to_csv(f'results/{test_case}_vector.csv')

    # sum all facades
    roof_df_cols = [col for col in combined_df.columns if not col.split('_')[1].isdigit()]
    roof_df = combined_df[roof_df_cols]
    facade_df = combined_df.drop(columns=roof_df_cols)
    facade_df.columns = facade_df.columns.str.split('_', n=1, expand=True)
    facade_sum = facade_df.groupby(level=0, axis=1).sum()
    new_column_names = []
    for col in facade_sum.columns:
        second_level_indices = facade_df[col].columns
        sum_of_indices = np.sum([int(idx) for idx in second_level_indices])
        new_name = f"{col}_{sum_of_indices}"
        new_column_names.append(new_name)
    facade_sum.columns = new_column_names
    final_df = pd.concat([roof_df, facade_sum], axis=1)
    final_df.to_csv(f'results/{test_case}_vector_combined_facade.csv')

C:\Users\vapeu\AppData\Local\Temp\ipykernel_48548\925072798.py:19: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  facade_sum = facade_df.groupby(level=0, axis=1).sum()
C:\Users\vapeu\AppData\Local\Temp\ipykernel_48548\925072798.py:19: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  facade_sum = facade_df.groupby(level=0, axis=1).sum()


In [44]:
building_roof_data, building_facade_data, building_facade_original = extract_result_entries(data)
facade_areas_df = transform_facade_areas(building_facade_data)
facade_areas_df_original = transform_facade_areas(building_facade_original)
roof_areas_df = transform_roof_areas(building_roof_data)
combined_df = pd.concat([roof_areas_df, facade_areas_df], axis=1)
sorted_columns = sorted(combined_df.columns, key=lambda x: int(x.split('_')[0]))
combined_df = combined_df[sorted_columns]

In [45]:
roof_df_cols = [col for col in combined_df.columns if not col.split('_')[1].isdigit()]
roof_df = combined_df[roof_df_cols]

In [46]:


# Rename the columns
result.columns = new_column_names
facade_sum.columns = facade_sum.columns + '_total'
final_df = pd.concat([roof_df, facade_sum], axis=1)

NameError: name 'result' is not defined

In [47]:
# Function to check if any 'area' is not equal to 0.0
def check_area_not_zero(data):
    for building, floors in data.items():
        for floor, details in floors.items():
            if details['area'] != 0.0:
                return True  # Found an area that is not 0.0
    return False  # No area was found that is not 0.0

# Check if any area is not equal to 0.0
has_non_zero_area = check_area_not_zero(building_roof_data)

# Display the result
print("Any area not equal to 0.0:", has_non_zero_area)

Any area not equal to 0.0: False
